In [2]:
import numpy as np
import os
import time
import gc
import cv2
import torch
import torchvision
import albumentations as A
import matplotlib
import matplotlib.pyplot as plt

from ensemble_boxes import *
from PIL import Image
from albumentations.pytorch.transforms import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (30,30)
matplotlib.rcParams['font.size'] = 14

In [3]:
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Running in {device}')

Running in cuda


In [4]:
def get_model(checkpoint_path):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 5)
    
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint)
    model = model.to(device)
    model.eval()
    del checkpoint
    gc.collect()
    
    return model


# models = [
#     get_model('../input/objectdetection/fold0_resnet50_BestModel.pth'), 
#     get_model('../input/objectdetection/fold1_resnet50_BestModel.pth'),
#     get_model('../input/objectdetection/fold2_resnet50_BestModel.pth'),
#     get_model('../input/objectdetection/fold3_resnet50_BestModel.pth'), 
#     get_model('../input/objectdetection/fold4_resnet50_BestModel.pth'),
#     get_model('../input/objectdetection/fold5_resnet50_BestModel.pth'),
# ]

model = get_model('weight\\fold1_resnet50_fpn_BestModel.pth')

In [5]:
transform = {
    'test': A.Compose([A.Resize(height=720, width=1280, p=1.0),
                      ToTensorV2(p=1.0)], p=1.0)
}

class CarTestDataset(Dataset):
    def __init__(self, image_root, depthmaps_path, transform=transform['test']):
        self.image_root = image_root
        self.transform = transform
        self.image_paths = [image_root + '/' + image_root.split('/')[-1].split('_')[-3] + f'_{i}_image.jpg' for i in range(901)]
        self.depthmaps = np.load(depthmaps_path)
        self.isleft = True if image_root.split('/')[-1].split('_')[-2]=='left' else False
        
    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = Image.open(image_path).convert('RGB')
        image = np.array(image).astype(np.float32) / 255.0
        
        tsfm = self.transform(**{'image':image})

        return tsfm['image'], self.depthmaps[index], self.isleft
    
    def __len__(self):
        return len(self.image_paths)

In [6]:
root_dicts = {
    'video_frames_images/ally_right_images': 'video_frames_images/ally_depth_mm.npy',
    'video_frames_images/enemy_left_images': 'video_frames_images/enemy_depth_mm.npy',
    'video_frames_images/ally_left_images': 'video_frames_images/ally_depth_mm.npy',
    'video_frames_images/enemy_right_images': 'video_frames_images/enemy_depth_mm.npy',
}

label2color = {
    1: (30,144,255),
    2: (0,255,255),
    3: (255,69,0),
    4: (240,128,128),
}

label2str = {
    1: 'Ally Robot',
    2: 'Ally Armor',
    3: 'Enemy Robot',
    4: 'Enemy Armor',
}

In [7]:
def get_distance(box, label, dm, isleft):
    '''
    Calculate the distance of the bounding box predicted from death map
    
    Input:
    box: list or numpy -> a single predicted box
    label: list or numpy -> a single predicted label
    isleft: boolean -> Is it the left side camera?
    
    Return:
    d: float -> The calculated distance
    '''
    if isleft:
        c = 50   
    if not isleft:
        if label == 1 or label == 3:
            c = round(175/366 * (box[3]-box[1]))
        else:
            c = round(175/45 * (box[3]-box[1]))
        
    xmid = round(box[0] + (box[2] - box[0])/2) + c
    ymid = round(box[1] + (box[3] - box[1])/2)
    
    d = dm[ymid-10:ymid, xmid:xmid+17][dm[ymid-10:ymid, xmid:xmid+17]!=0]
    return 0.0 if not len(d) else d.mean()
       
def draw_info(np_image, boxes, labels, isleft):
    '''
    Draw predicted bounding box and distance on the image
    
    Input:
    np_image: numpy -> an image in the numpy form
    boxes: numpy or list -> Predicted bounding boxes of a single image
    labels: numpy or list -> Predicted labels of a single image
    isleft: boolean -> Is it the left side camera?
    
    Return:
    np_image: numpy -> An image with info on it
    '''
    np_image = np_image.copy()
    
    for box, label in zip(boxes, labels):
        color = label2color[label]
        name = label2str[label]
        distance = get_distance(box, label, dm, isleft)
        text = name + f'({distance/1000:.3f}m)'
        
        cv2.rectangle(np_image, (box[0], box[1]), (box[2], box[3]), color, 3)
        cv2.rectangle(np_image, (box[0], box[1] - 30), (box[0] + round(len(text)/19 * 230), box[1]), color, -1)
        cv2.putText(np_image, text, (box[0], box[1] - 5), cv2.FONT_HERSHEY_PLAIN, 1.25, (0,0,0), 2)
    np_image = cv2.cvtColor(np_image, cv2.COLOR_BGR2RGB)
    return np_image
    
def get_valid_prediction(boxes: np.ndarray, scores: np.ndarray, labels: np.ndarray, threshold=0.75):
    '''
    Get predictions which satisfied the conditions. We will choose the boxes with confidence>0.75, and set the
    robot's box as a reference, then calculate the ymin threshold for armor's boxes, as all armor should be at the bottom half of 
    the robot's box
    
    Input:
    boxes: numpy -> Predicted boxes on an single image
    scores: numpy -> Predicted scores on an single image
    labels: numpy -> Predicted labels on an single image
    
    Return:
    valid_boxes: list
    valid_scores: list
    valid_labels: list
    '''
    boxes = boxes[scores > threshold]
    labels = labels[scores > threshold]
    scores = scores[scores > threshold]
    
    robot_boxes = boxes[np.logical_or(labels==1, labels==3)]
    robot_labels = labels[np.logical_or(labels==1, labels==3)]
    robot_scores = scores[np.logical_or(labels==1, labels==3)]
    
    armor_boxes = boxes[np.logical_or(labels==2, labels==4)]
    armor_labels = labels[np.logical_or(labels==2, labels==4)]
    armor_scores = scores[np.logical_or(labels==2, labels==4)]
    
    if len(robot_boxes) > 0:
        ymin_ref = robot_boxes[:, 1].mean()
        ymax_ref = robot_boxes[:, 3].mean()

        armor_ymin_threshold = ymin_ref + (ymax_ref - ymin_ref) / 2

        armor_labels = armor_labels[armor_boxes[:, 1] >= armor_ymin_threshold]
        armor_scores = armor_scores[armor_boxes[:, 1] >= armor_ymin_threshold]
        armor_boxes = armor_boxes[armor_boxes[:, 1] >= armor_ymin_threshold]

    valid_boxes = []
    valid_labels = []
    valid_scores = []

    for box, score, label in zip(robot_boxes, robot_scores, robot_labels):
        valid_boxes.append(box)
        valid_scores.append(score)
        valid_labels.append(label)

    for box, score, label in zip(armor_boxes, armor_scores, armor_labels):
        valid_boxes.append(box)
        valid_scores.append(score)
        valid_labels.append(label)
    
    return valid_boxes, valid_scores, valid_labels

@torch.no_grad()
def make_prediction(image):
    '''
    Make prediction on a single image
    
    Input:
    image: tensor with shape (channels, height, width) #without batch_size!!!!
    
    Return:
    prediction: dictionary which contain (boxes, scores, labels)
    np_image: a numpy image of the input image
    '''
    np_image = np.ascontiguousarray(image.permute(1,2,0).mul(255).byte().cpu())
    prediction = model(image.unsqueeze(0).to(device))
    return prediction, np_image

def run_wbf(prediction, image_max_size, weights=None, iou_thr=0.55, skip_box_thr=0.75):
    boxes = [(prediction[0]['boxes'].clip(min=0, max=image_max_size-1)/(image_max_size-1)).tolist()]
    scores = [prediction[0]['scores'].tolist()]
    labels = [prediction[0]['labels'].tolist()]

    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = (boxes * (image_max_size-1)).clip(min=0, max=image_max_size-1).astype(np.int32)
    return boxes, scores, labels

In [78]:
start = time.time()
for image_root, dm_path in root_dicts.items():
    test_ds = CarTestDataset(image_root=image_root, depthmaps_path=dm_path)
    
    video_filename = 'result_videos/' + image_root.split('/')[-1] + '.avi'
    size = (1280, 720)

    out = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
    
    for i, ds in enumerate(test_ds):
        image, dm, isleft = ds

        prediction, np_image = make_prediction(image)
        
        boxes, scores, labels = run_wbf(prediction=prediction, image_max_size=1280)
        boxes, scores, labels = get_valid_prediction(boxes, scores, labels)
        
        np_image = draw_info(np_image, boxes, labels, isleft)

        out.write(np_image)
    
    out.release()

    print(f'{video_filename} is done!!\n')
print('Average FPS:', f'{(901 * 4 / (time.time()-start)):.4f}')

ally_right_images.avi is done!!

enemy_right_images.avi is done!!

ally_left_images.avi is done!!

enemy_left_images.avi is done!!

Average FPS: 7.8801
